In [2]:
import requests
from bs4 import BeautifulSoup

baseUrl = 'https://www.wine.com'

page = requests.get(f'{baseUrl}/list/wine/red-wine/7155-124?showOutOfStock=true')

soup = BeautifulSoup(page.text, 'html.parser')


In [3]:
wineList = soup.find('ul', class_='listGridLayout_list')
wineElements = wineList.find_all('li', class_='listGridLayout_listItem')

In [48]:
for wineElem in wineElements[:10]:
    trials = 0
    while trials < 5:
        try:
            outOfStock = wineElem.find('div', class_='productUnavailable')

            wineInfo = { 'color': 'red' }
            a = wineElem.find('a', class_='event_productClick', href=True)
            
            wineUrl = f"{baseUrl}{a['href']}"

            winePage = requests.get(wineUrl)
            wineSoup = BeautifulSoup(winePage.text, 'html.parser')

            # Name and vintage  
            name = wineSoup.find('h1', class_='pipName').text
            vintage = name[-4:]
            name = name[:-4].strip()
            wineInfo['name'] = name
            wineInfo['vintage'] = vintage

            # Varietal and Origin
            divOrigin = wineSoup.find('div', class_='pipOrigin')

            spanVarietal = divOrigin.find('span', class_='prodItemInfo_varietal')
            varietal = spanVarietal.find('a', href=True).text
            wineInfo['varietal'] = varietal

            spanOrigin = divOrigin.find('span', class_='prodItemInfo_originText')
            origin = spanOrigin.find('a', href=True).text
            wineInfo['origin'] = origin        

            # Ratings
            ulRatings = wineSoup.find('ul', class_='pipProdWineRatings')
            if ulRatings:
                ratings = []
                ratingsList = ulRatings.find_all('li', class_='pipProdWineRatings_item')
                for ratingElem in ratingsList:
                    ratingName = ratingElem.contents[2].strip() + " " + ratingElem.contents[4].strip()
                    
                    ratingValue = ratingElem.find('span', class_='pipProdWineRatings_rating').text

                    rating = {'name': ratingName, 'rating': int(ratingValue)}

                    ratings.append(rating)

                wineInfo['ratings'] = ratings

            if not outOfStock:
                productDetails = wineSoup.find('section', class_='pipProdDetails')
                titles = productDetails.find_all('div', class_='pipProdDetails_title')
                values = productDetails.find_all('div', class_='pipProdDetails_name')
                for i, title in enumerate(titles):
                    if title.text.strip() == 'Size':
                        wineInfo['size'] = values[i].text[:-2]
                    elif title.text.strip() == 'Producer':
                        wineInfo['producer'] = values[i].text
                        wineInfo['name'] = wineInfo['name'].replace(wineInfo['producer'], '').strip()
                
                divImage = wineSoup.find('div', class_='pipProdThumbs')
                images = []
                for image in divImage.find_all('img'):
                    imageUrl = image['src'].split('/')[-1]
                    images.append('https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/' + imageUrl)

                priceDiv = wineSoup.find('div', class_='productPrice')
                price = priceDiv.find('meta', itemprop='price')['content']
                wineInfo['price'] = float(price)
            else:
                size = wineSoup.find('span', class_='prodAlcoholVolume_text').text
                wineInfo['size'] = size

                divImage = wineSoup.find('div', class_='pipThumbs')
                images = []
                for image in divImage.find_all('img'):
                    imageUrl = image['src'].split('/')[-1]
                    images.append('https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/' + imageUrl)
                
                priceDiv = wineSoup.find('div', class_='prodOffer')
                price = priceDiv.find('meta', itemprop='price')['content']
                wineInfo['price'] = float(price)
            
            wineInfo['image'] = images

            print(wineUrl)
            for key, value in wineInfo.items():
                print(f"{key}: {value}")
            print()
            
            break
        except:
            trials += 1
            continue
    else:
        print(f"FAIL {wineUrl}")

https://www.wine.com/product/altesino-brunello-di-montalcino-2017/877499
color: red
name: Brunello di Montalcino
vintage: 2017
varietal: Sangiovese
origin: Montalcino, Tuscany, Italy
ratings: [{'name': 'James Suckling', 'rating': 95}, {'name': 'Robert Parker', 'rating': 94}, {'name': 'Wine Spectator', 'rating': 92}, {'name': 'Wine Enthusiast', 'rating': 90}]
producer: Altesino
size: 750
price: 74.99
image: ['https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/hhcpugypymo1t9rosdye.jpg', 'https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/z89hjnmoryon9ij12ew7.jpg']

https://www.wine.com/product/le-cellier-des-princes-cotes-du-rhone-la-couronne-du-prince-2018/1079083
color: red
name: Cotes du Rhone La Couronne du Prince
vintage: 2018
varietal: Rhone Red Blends
origin: Cotes du Rhone, Rhone, France
ratings: [{'name': 'James Suckling', 'rating': 91}]
producer: Le Cellier des Princes
size: 750
price: 15.98
image: ['https://www.wine.c

In [33]:
text = 'Antinori Badia a Passignano Chianti Classico Gran Selezione 2018'

print(text.replace('Antinori', ''))

 Badia a Passignano Chianti Classico Gran Selezione 2018
